In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pylab as pylab
import pandas as pd
from operator import itemgetter
pylab.rcParams['figure.figsize'] = 10, 10
plt.rc("font", size=8)

plt.rc('xtick', color='k', labelsize='medium', direction='in')
plt.rc('xtick.major', size=8, pad=12)
plt.rc('xtick.minor', size=8, pad=12)

plt.rc('ytick', color='k', labelsize='medium', direction='in')
plt.rc('ytick.major', size=8, pad=12)
plt.rc('ytick.minor', size=8, pad=12)

# Regression

## Table Of Content: <a id='toc'></a>

* [Introduction](#intro)

* [Data exploration](#Exploration)


* [Linear regression](#Linear-Regression)

    * [ML approach to linear regression - diabetes data](#LR-1)
    * [Adding regularization to the linear regression](#LR-regul)
    
* [KNN](#KNN)




* [Random Forest](#rf-r)



* [Exercise](#exo)

* [Appendices](#APPENDIX)

    * [SVM](#SVM)
    * [Gradient Boosting](#Gradient)


# Introduction : re-framing the target <a id='intro'></a>

In that part we will use all the algorithms seen previously but in the case of a regression rather than a classification.

**The difference is that now our target vector (i.e. what we want to predict) is not a class (a discrete variable) but rather a number that can take a continuity of values.**

The algorithms are going to use very similar methods than the ones used in classification (if there are particularities between using a method as a classifier and using the same method as a regressor I will mention it).

**What's mainly going to change is the loss function used to train your model and the scoring used to evaluate this model.**

**For classification** we saw two main types of **loss function**:

- Cross entropy (negative loglikelihood) : all the classifiers except SVM and KNN (because KNN doesn't learn through a loss function). Note : GINI impurity is a form of entropy.

- Hinge loss function : SVM classifier.

And plenty of **scoring systems**, but at least two in depth:

- Accuracy

- Area under the ROC curve.

Note that we could have used the value of the minimized loss function as a score. But for classification problems since the loss function can be different from one model to another, the values of the loss function can be hard to compare and you will prefer a scoring method independent of your loss function.

### In regression problems the loss functions and the scoring are metrics that you probably have seen before and so are more intuitive.

The **two main loss functions that you will minimize while training your model are** :


- Mean square error/ quadratic loss/ L2 loss **(MSE)** : $\frac{\Sigma^{n}_{i=1}(y_{i}-\hat{y}_{i})^{2}}{n}$

- Mean absolute error/ L1 loss **(MAE)** : $\frac{\Sigma^{n}_{i=1}|y_{i}-\hat{y}_{i}|}{n}$

Where 
 * $n$ are the number of training examples
 * $i$ the $i$-th training example in the data set
 * $y_{i}$ the ground truth target value for that $i$-th example and 
 * $\hat{y_{i}}$ the estimated value of the target for the $i$-th example.

**Scoring to consider** here can actually be the value of the minimized loss function used (MSE or MAE) or other scoring metrics like :

- **R squared** : $1-\frac{\Sigma^{n}_{i=1}(y_{i}-\hat{y}_{i})^{2}}{\Sigma^{n}_{i=1}(y_{i}-\bar{y}_{i})^{2}}$

Proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model

- **Adjusted R squared** : $1-(1-\frac{\Sigma^{n}_{i=1}(y_{i}-\hat{y}_{i})^{2}}{\Sigma^{n}_{i=1}(y_{i}-\bar{y}_{i})^{2}}\frac{n-1}{n-(k+1)})$ , with k the degree of freedom.

Adjusted R-squared is similar to R-squared, but it takes into account the number of variables in the model.

- **Max error** : $max(|y_{i}-\hat{y}_{i}|)$

- **Variance explained** : $1-\frac{Var(\{y_{i}-\bar{y}_{i}\}_{i})}{Var(\{y_{i}\}_{i})}$

---

We will use a DNA methylation dataset from [Naue etal. 2017](https://www.sciencedirect.com/science/article/pii/S1872497317301643?via%3Dihub), as prepared for another [regression tutorial from the GTN](https://training.galaxyproject.org/training-material/topics/statistics/tutorials/regression_machinelearning/tutorial.html).

It consists in measurements of the methylation levels of 13 biomarkers to try to predict age.

The train and the test data have already been separated in two separate tables: 

In [ ]:
df_meth = pd.read_table( "https://zenodo.org/record/2545213/files/train_rows.csv" )
df_meth.head()

In [ ]:
df_meth_test = pd.read_table("https://zenodo.org/record/2545213/files/test_rows_labels.csv")
df_meth_test.head()

In [ ]:
df_meth.shape

[Back to ToC](#toc)

# A little bit on data exploration for regression <a class="anchor" id="Exploration"></a>

let's check how the different features and the target interact with each others.

In [ ]:
df_meth_corr = df_meth.corr()

sns.clustermap(df_meth_corr,
               figsize=(10,10),
               z_score=None,
               row_cluster=True,
               col_cluster=True,
               method='ward',
               cmap='coolwarm',vmax=1,vmin=-1, 
               annot=True, annot_kws={"size": 13},cbar_kws={"label": 'Pearson\ncorrelation'})
## sns allows you to do a hierarchical clustering that simply
plt.show()

We see that ELOVL2_6 has a very high correlation with age. Perhaps a model with just this marker would be enough?

In [ ]:
sns.scatterplot(df_meth , x = 'ELOVL2_6' , y = 'Age')

[Back to the ToC](#toc)

# Linear regression <a class="anchor" id="Linear-Regression"></a>

We will start with a reduced dataset for visualization purposes.

In [ ]:
from sklearn.model_selection import cross_val_score
X_ELOVL2_6 = X_meth.loc[:,["ELOVL2_6"]]

## to plot nice curves, it is interesting to get an ordered version of X
## points sorted by their values of ELOVL2_6
ELOVL2_6_sorted = X_ELOVL2_6.sort_values(by = "ELOVL2_6")

Here, as we will build a simple linear model with a single feature, we actually have no hyper-parameter at this point.

So we can just build the model with `LinearRegression`:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import root_mean_squared_error

## building and fitting model
lr=LinearRegression()
lr.fit(X_ELOVL2_6,y_meth)

## prediction and scoring
y_predict=lr.predict(X_ELOVL2_6)

R2=r2_score(y_meth,y_predict)
RMSE=root_mean_squared_error(y_meth,y_predict)


## plotting

plt.plot(X_ELOVL2_6.ELOVL2_6,y_meth,'ko',label='Data')

plt.plot(ELOVL2_6_sorted.ELOVL2_6, 
         lr.predict( ELOVL2_6_sorted ) ,
         'r-.',label='Predicted', linewidth = 3)

plt.legend(loc='best',fontsize=10)

plt.title('R2={0:.2f}, RMSE={1:.2f}'.format(R2,RMSE))

plt.xlabel('X')
plt.ylabel('y')
plt.show()

If we want to compute a cross-validated score for this simple model we can use `cross_val_score`

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_r2 = cross_val_score( LinearRegression() ,  
                               X_ELOVL2_6 , y_meth , scoring='r2', cv = 5 )

## NB: the built-in scorers want to have a score which should be maximized,
##     so instead of RMSE, it compute negative RMSE
cross_val_RMSE = cross_val_score( LinearRegression() , 
                                 X_ELOVL2_6 , y_meth , scoring='neg_root_mean_squared_error', cv = 5 )

print(f"cross-validated R2: {cross_val_r2.mean():.2f}")
print(f"cross-validated RMSE: {-1 * cross_val_RMSE.mean():.2f}")


The model seems OK-ish, but if you have some experience with linear models you may see something peculiar.

It becomes more apparent when you look at residuals (ie. error from the model):


In [ ]:
sns.regplot( x = X_ELOVL2_6.ELOVL2_6 , y = residuals , lowess=True)
plt.axhline(y=0, color='black')
plt.ylabel("residuals")

The pattern of residuals may be indicative of a higher order relationship between the marker and age.

But if that's the case, what is an appropriate degree for this model?

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

fig,ax = plt.subplots()

## plotting
ax.plot(X_ELOVL2_6.ELOVL2_6,y_meth,'ko',label='Data')

for i,degree in enumerate([2,3,40]):



    X = PolynomialFeatures(degree = degree).fit_transform(StandardScaler().fit_transform(X_ELOVL2_6))
    Xo = PolynomialFeatures(degree = degree).fit_transform(StandardScaler().fit_transform(ELOVL2_6_sorted) )
    y = y_meth 

    lr = LinearRegression()
    lr.fit( X,y )

    ## prediction and scoring
    y_predict=lr.predict(X)

    ax.plot(ELOVL2_6_sorted.ELOVL2_6, 
            lr.predict( Xo ) ,
            '-.',label=f'degree: {degree}', linewidth = 3)

    
    R2=r2_score(y,y_predict)
    RMSE=root_mean_squared_error(y,y_predict)
    
    print(f'degree: {degree} - R2={R2:.2f}, RMSE={RMSE:.2f}')
    
ax.legend(loc='best')
ax.set_xlabel('ELOVL2_6')
ax.set_ylabel('age')



* which degree relationship is appropriate then ?
 * what of the other biomarkers ?

You guessed it, we come back again to hyper-parameters we have to evaluate in order to find the most appropriate model.

In [ ]:
%%time
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


pipeline_lr=Pipeline([('scalar',StandardScaler()),
                      ('poly',PolynomialFeatures(include_bias=False)),
                      ('model',LinearRegression())])


# define the hyperparameters you want to test with their range to be tested
grid_values = {'poly__degree': [1,2,3,4,5]}

# Feed them to GridSearchCV with the right score (R squared)
grid_lr = GridSearchCV(pipeline_lr, 
                       param_grid = grid_values, 
                       scoring=['r2','neg_root_mean_squared_error'],
                       refit = 'r2',
                       cv = 5,
                       n_jobs = -1)

grid_lr.fit(X_ELOVL2_6,y_meth)


print(f'Grid best score (r2): {grid_lr.best_score_:.3f}' )
print( 'Grid best parameter (max. r2): ')
for k,v in grid_lr.best_params_.items():
    print( f'\t{k} -> {v}' )

In [ ]:
df_res = pd.DataFrame( grid_lr.cv_results_ )
df_res[ ["param_poly__degree","mean_test_r2","mean_test_neg_root_mean_squared_error"] ]

So it seems that increasing power to 3 increases (at least a bit) our predictive power here.

But what about the other predictors? 



[Back to the ToC](#toc)

## ML approach to linear regression - methylation data <a id='LR-1'></a>

As we add other predictors, it can be good to also add penalization terms.

To enable this we switch from `LinearRegression` to `SGDRegressor` which uses Stochastic Gradient Descent to find the optimal weigths for the penalized score.

In [ ]:
%%time
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor


ppl_meth=Pipeline([('scalar',StandardScaler()),
                   ('poly',PolynomialFeatures(include_bias=False)),
                   ('model',SGDRegressor( penalty = 'elasticnet' ))])


# the interaction_only parameter of PolynomialFeature 
# lets us test a case with interaction terms but no higher order terms.
grid_values = {'poly__degree': [1,2,3],
               'poly__interaction_only':[False,True], 
               'model__alpha' : np.logspace( -2,2,20 ),
               'model__l1_ratio' : np.linspace(0.0,1.0,11)
              }

# Feed them to GridSearchCV with the right score (R squared)
grid_meth = GridSearchCV(ppl_meth,
                         param_grid = grid_values, 
                         scoring=['r2', 'neg_root_mean_squared_error'],
                         refit = 'r2',
                         cv = 5,
                        n_jobs = -1)

grid_meth.fit(X_meth, y_meth)


print(f'Grid best score (grid_meth.scoring): {grid_meth.best_score_:.3f}' )
print( 'Grid best parameter (max. r2): ')
for k,v in grid_meth.best_params_.items():
    print( f'\t{k} -> {v}' )

In [ ]:
poly = grid_meth.best_estimator_['poly']
LR = grid_meth.best_estimator_['model']

def pow2name(power, fnames=X_meth.columns ):
    n = []
    for i,j in enumerate(power):
        if j>0:
            if j == 1:
                n.append( fnames[i] )
            else:
                n.append( fnames[i]+'^'+str(j) )
    return "_x_".join(n)
    
sorted_list=sorted( zip( map( pow2name , poly.powers_) , LR.coef_ ) ,key=itemgetter(1),reverse=True)
print('model importances')
for f,w in sorted_list:
    if w != 0:
        print("{:>30}\t{:.2f}".format(f,w))
    

In [ ]:
y_predict = grid_meth.best_estimator_.predict( X_meth)

R2=r2_score(y_meth,y_predict)
RMSE=root_mean_squared_error(y_meth,y_predict)


plt.plot(y_meth,y_predict,'ro',label='Predicted', linewidth = 3)
plt.title('R2={0:.2f}, RMSE={1:.2f}'.format(R2,RMSE))
plt.axline((20, 20), slope=1, color="black", linestyle='dashed')
plt.xlabel('Age - observed')
plt.ylabel('Age - predicted')
plt.show()

In [ ]:
df_res

In [ ]:



df_res = pd.DataFrame(grid_meth.cv_results_)
print("R-squared")
print( df_res.groupby(['param_poly__degree','param_poly__interaction_only'])[ "mean_test_r2" ].max() )

print("neg RMSE")
print( df_res.groupby(['param_poly__degree','param_poly__interaction_only'])["mean_test_neg_root_mean_squared_error"].max())

[Back to the ToC](#toc)

# KNN <a class="anchor" id="KNN"></a>

You can also use the KNN algorithm for regression.
Again you get the K nearest neighbors of the point you want to predict but this time instead of a vote it does a local average of the neighbours value (again weighted or not by their distance to your query point).

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
help( KNeighborsRegressor )

**micro-exercise:** complete the code below to train a KNN regressor model for the methylation data. Optimize the `n_neighbors` and `weights` hyper-parameters:

In [ ]:
%%time
from sklearn.neighbors import KNeighborsRegressor


ppl_meth_KNN=Pipeline([('scalar',StandardScaler()),
                   ('poly',PolynomialFeatures(include_bias=False)),
                   ('model', ... )])



grid_values = {'poly__degree': [1,2,3],
               'poly__interaction_only':[False,True], 
               ... ### change things here
              }


grid_meth_KNN = GridSearchCV(ppl_meth_KNN,
                             param_grid = grid_values, 
                             scoring=['r2', 'neg_root_mean_squared_error'],
                             refit = 'r2',
                             cv = 5)

grid_meth_KNN.fit(X_meth, y_meth)


print(f'Grid best score (grid_meth.scoring): {grid_meth_KNN.best_score_:.3f}' )
print( 'Grid best parameter (max. r2): ')
for k,v in grid_meth_KNN.best_params_.items():
    print( f'\t{k} -> {v}' )

In [ ]:
## getting the corresponding RMSE require some legwork
cv_res = pd.DataFrame( grid_meth_KNN.cv_results_ )
cv_res.loc[ cv_res.rank_test_r2 == 1 , 'mean_test_neg_root_mean_squared_error']

So we see that the model we get from KKN is not bad at all **but** the interpretability is lacking in this model.

> KNN regressor is not really good at predicting sparse neigborhoods (the predictions for the extreme data points are not accurate).

<br>

[Back to the ToC](#toc)

<br>

## Random Forest <a class="anchor" id="rf-r"></a>

From the standpoint of tree, the only difference is that now, instead of the entropy or Gini criterion, **the decision which variable to use at any node is made using a regression metric**, such as squared error for example.

For example, consider this example of [regression with a single tree](https://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html), adapted from the sklearn website:

In [ ]:
from sklearn.tree import DecisionTreeRegressor
# Create a random dataset
rng = np.random.RandomState(1)
X = np.sort(5 * rng.rand(80, 1), axis=0)
y = np.sin(X).ravel()
y[::5] += 3 * (0.5 - rng.rand(16)) # adding additional noise to some of the points

# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(X, y)
regr_2.fit(X, y)

# Predict
X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)

# Plot the results
plt.figure(figsize = (14,6))
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(X_test, y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test, y_2, color="yellowgreen", label="max_depth=5", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()


In [ ]:
from sklearn.tree import plot_tree
fig,ax = plt.subplots(figsize=(10,5))
plot_tree( regr_1 , 
          ax=ax , fontsize=10 , filled=True , impurity=False , precision=3)
ax.set_title('best single decision tree')

Of course with a single tree you do not get very far, unless the tree becomes absolutely huge. 

But with a random forest you can aggregate the estimate from many trees to get somewhere nice.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RFReg = RandomForestRegressor(n_estimators=10 )
RFReg.fit(X, y)

# Predict
X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
y_1 = regr_1.predict(X_test)
y_rf = RFReg.predict(X_test)

# Plot the results
plt.figure(figsize = (14,6))
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(X_test, y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test, y_rf, color="yellowgreen", label="RF", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()



With a bit of leg-work, we can even grab the inidividual trees predictions to build an interval around the random forest prediction: 

In [ ]:

## collecting prediction from all individual trees in a big list
y_pred = []
x_pred = []
for tree in RFReg.estimators_ :
    y_pred += list( tree.predict(X_test) )
    x_pred += list(X_test[:,0])


plt.figure(figsize = (14,6))
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(X_test, y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test, y_rf, color="yellowgreen", label="RF", linewidth=2)
sns.lineplot(x=x_pred , y=y_pred , color="yellowgreen" , errorbar = 'sd') 
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()


Let's try on the methylation data:

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor

## when it comes to criterion, we can now choose:
# * “squared_error” (default) for the mean squared error, minimizes the L2 loss
#                                           using the mean of each terminal node,
# * “friedman_mse”, which uses mean squared error with Friedman’s improvement score for potential splits
# * “absolute_error” for the mean absolute error, which minimizes the L1 loss
#                                           using the median of each terminal node,
# * “poisson” which uses reduction in Poisson deviance to find splits.
#
# let's try squared error and absolute error

grid_values = {'criterion': ['squared_error' , 'absolute_error'],
               'n_estimators':[1000], 
               'max_depth':[6,8,10],
               'min_samples_split':np.arange(2,23,5)}

grid_RF = GridSearchCV(RandomForestRegressor(),
                                param_grid = grid_values, 
                                scoring=['r2', 'neg_root_mean_squared_error'],
                                refit = 'r2',
                                n_jobs=-1,
                                cv=5)

grid_RF.fit(X_meth, y_meth)


print('Grid best score (r2): ', grid_RF.best_score_)
print('Grid best parameter (max. r2): ')

for k,v in grid_RF.best_params_.items():
    print( f'\t{k} -> {v}' )

In [ ]:
feature_importance=grid_RF.best_estimator_.feature_importances_


sorted_features=sorted( list( zip( X_meth.columns , feature_importance ) ),key=itemgetter(1),reverse=True)

print('Features sorted per importance in discriminative process')
for f,w in sorted_features:
    print('{:>20}\t{:.3f}'.format(f,w))

Tree-based techniques are interesting because:
 * they do not necessitate scaling
 * they give interpretable models and results
 * they model arbitrary non-linear problems
 
However as you have seen they tend to take longer to train...

[Back to ToC](#toc)

# Exercise:  <a class="anchor" id="exo"></a>

We go back to the potato dataset we introduced in notebook 3.

> the model can take even longer than before to train. Be mindful of the parameter space you specify. For instance, if you use a polynomial : do you really want a 3rd degree polynomial exploring all combinations of 15000 genes ? How many parameters would that represent?
 Again, `SelectKBest`, and/or a preliminary PCA are your friend (use scores adapted to regression)

Try linear regression (with different degrees if you wish) and gradient boosting tree. What do you learn about Flesh Color?

In [ ]:
file_metadata = "../data/potato_data.phenotypic.csv"
file_data = "../data/potato_data.transcriptomic.top200norm.csv"

df = pd.read_csv( file_metadata , index_col=0 )
dfTT = pd.read_csv( file_data , index_col=0)

X = dfTT
X.head()

In [ ]:
y = df["Flesh Colour"]
y.describe()

separate train and test sets

In [ ]:
# %load -r 1-6 solutions/solution_04_potato.py

finding the best linear model

In [ ]:
# %load -r 8-36 solutions/solution_04_potato.py

finding the best KNN

In [ ]:
# %load -r 37-63 solutions/solution_04_potato.py

finding the best random forest

In [ ]:
# %load -r 64-85 solutions/solution_04_potato.py

choosing the best method 

In [ ]:
# %load -r 86-90 solutions/solution_04_potato.py

In [ ]:
# %load -r 91-105 solutions/solution_04_potato.py

# Extra exercise

You can attempt the same thing with the AML dataset we used in the unsupervised learning:

In [ ]:
df_aml=pd.read_excel('../data/gene_drug_sens.xlsx')
X_aml=df_aml[df_aml.columns[9:]] 
y_aml = df_aml.auc #  <-- this is the target metric
X_aml.head()

[Back to ToC](#toc)

# Appendices <a class="anchor" id="APPENDIX"></a>

[Back to the ToC](#toc)

# SVM <a class="anchor" id="SVM"></a>

In SVM regression the algo is almost the same as classifier SVM with one big difference. In classification we were looking at the hyperplane that was as far as possible from our support vectors.
In regression you look for the **hyperplanes that are as close as possible from your support vector**.

> NB: **Support vectors** are data points that are closer to the hyperplane and influence the position and orientation of the hyperplane. 


In [ ]:
from sklearn.svm import SVR

svr=SVR()

pipeline_svr=Pipeline([('scalar',StandardScaler()),('model',svr)])

from sklearn.model_selection import GridSearchCV

# define the hyperparameters you want to test with their range
# kernel: kernel type to be used in the algorithm (function that takes low dimensional input space 
# and transforms it to a higher dimensional space)
# degree: degree of the polynomial kernel function
# epsilon: parameters that controls which data points contribute to regularization
# C: inverse of regularization strength
grid_values = [{"model": [SVR(kernel='linear')],
                 "model__C":np.logspace(-2, 2, 10)},
                {"model": [SVR(kernel='rbf')],
                 "model__gamma": np.logspace(-2,1,10)},
                {"model": [SVR(kernel='poly')],
                 "model__C":np.logspace(-2, 2, 10),
                 "model__degree":np.arange(2,10,1)}]

# Feed them to GridSearchCV with the right score (R squared)
grid_svr_acc = GridSearchCV(pipeline_svr, param_grid = grid_values, scoring='r2')

grid_svr_acc.fit(X_diabetes_train, y_diabetes_train)

y_decision_fn_scores_acc=grid_svr_acc.score(X_diabetes_test,y_diabetes_test)


print('Grid best parameter (max. r2): ', grid_svr_acc.best_params_)
print('Grid best score (r2): ', grid_svr_acc.best_score_)
print('Grid best parameter (max. r2) model on test: ', y_decision_fn_scores_acc)

In [ ]:
best_kernel = grid_svr_acc.best_estimator_.steps[1][1].kernel
if best_kernel=='linear':
    
    svr = grid_svr_acc.best_estimator_.steps[1][1]
    
    featureW = pd.DataFrame( {'feature' : diabetes['feature_names'],
                                'weight': svr.coef_.flatten()} )

    # sort them by absolute value
    featureWsorted = featureW.sort_values(by=['weight'] , 
                                      ascending=False , 
                                      key=lambda col : col.abs())

    # get the non-null ones
    print('Features sorted per importance:')
    print( featureWsorted.loc[ featureWsorted["weight"] !=0 ] )

    

[Back to the ToC](#toc)

# Gradient boosting tree <a class="anchor" id="Gradient"></a>

Gradient boosted is made for regression (you saw that already)! It is even simpler than what we saw before since you don't have to do all those transformations and just go for minimizing the MSE loss function. So let's check it out.

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingRegressor

# define the hyperparameters you want to test with their range
grid_values = {'learning_rate':np.logspace(-1,0,5),
               'n_estimators':np.arange(25,101,25), 
               'max_depth':[2,3,4],
               'min_samples_split':[10,20],
               'min_samples_leaf':[2,5]}

# Feed them to GridSearchCV with the right score (R squared)
grid_tree = GridSearchCV(GradientBoostingRegressor(), param_grid = grid_values, scoring='r2')

grid_tree.fit(X_diabetes_train, y_diabetes_train)

y_decision_fn_scores=grid_tree.score(X_diabetes_test,y_diabetes_test)


print('Grid best parameter (max. r2): ', grid_tree.best_params_)
print('Grid best score (r2): ', grid_tree.best_score_)
print('Grid best parameter (max. r2) model on test: ', y_decision_fn_scores)

In [ ]:
RFC = grid_tree.best_estimator_
    
featureW = pd.DataFrame( {'feature': diabetes['feature_names'],
                        'weight':RFC.feature_importances_} )

# sort them by absolute value
featureWsorted = featureW.sort_values(by=['weight'] , 
                                      ascending=False , 
                                      key=lambda col : col.abs())

# get the non-null ones
print('Features sorted per importance:')
print( featureWsorted.loc[ featureWsorted["weight"] !=0 ] )